In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from time import time
from tensorflow.python.keras.callbacks import TensorBoard
import tensorflow as tf
import keras  
from keras.applications import DenseNet121
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers
from tensorflow.keras import Input
from tensorflow.keras import layers, Model

In [ ]:
input_img = Input(shape=(600, 600, 3))

print(input_img)
x = layers.Conv2D(128, kernel_size = 3, strides = 1,  activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
print(x)
x = layers.Conv2D(64, kernel_size = 3, strides = 1, activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
print(x)
x = layers.Conv2D(32, kernel_size = 3, strides = 1, activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)
print(encoded)

x = layers.Conv2DTranspose(64, (3, 3), strides = 2, activation='relu', padding='same')(encoded)
print(x)
x = layers.Conv2DTranspose(128, (3, 3), strides = 2, activation='relu', padding='same')(x)
print(x)
decoded = layers.Conv2DTranspose(3, (3, 3), strides = 2, activation='relu', padding='same')(x)
print(decoded)
autoencoder = Model(input_img,decoded)

In [ ]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Loading training data
import os
trainfalsedirpath = 'Folds/3folds/fold1/Train/False'
trainfalsedir = os.listdir(trainfalsedirpath)


xtrain = np.zeros([len(trainfalsedir),600,600,3])

for k,v in enumerate(trainfalsedir):
      xtrain[k] = img_to_array(load_img(trainfalsedirpath + '/' + v))

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1/255.)

In [ ]:
bs = 34

In [ ]:
numit = NumpyArrayIterator(
    x = xtrain,
    y = None,
    image_data_generator = train_datagen,
    batch_size = bs,
    shuffle = True,
    seed = 123,
    
)

In [ ]:
#Loading validation data
valfalsedirpath = 'Folds/3folds/fold1/Validation/False'
valfalsedir = os.listdir(valfalsedirpath)


xval = np.zeros([len(valfalsedir),600,600,3])

for k,v in enumerate(valfalsedir):
      xval[k] = img_to_array(load_img(valfalsedirpath + '/' + v))/255.

In [ ]:
from datetime import datetime
f = 1

In [ ]:
noise_factor = 0.5
lowest_val_loss = float('inf')
epochs = 50

seedarraypresent = 'seedarray.npy' in os.listdir()
if seedarraypresent:
    seedarray = np.load('seedarray.npy')
else:
    np.random.seed(123)
    seedarray = np.random.randint(1000, size = epochs)
    np.save('checkpointsandlogs/deepautoencoder/False_fold:{}_'.format(f) + 'seedarray.npy', seedarray)

for i in range(50):
    print('Epoch:{}'.format(i))
    if 'numit' in dir():
        del numit
    
    numit = NumpyArrayIterator(
    x = xtrain,
    y = None,
    image_data_generator = train_datagen,
    batch_size = bs,
    shuffle = True,
    seed = seedarray[i],   
    )
    
    for samples in numit:
        samples_noisy = samples.copy()
        samples_noisy+=(noise_factor * np.random.normal(loc=0.0, scale=1.0, size = samples_noisy.shape))
        autoencoder.fit(
            samples_noisy,
            samples,
            batch_size = 34,
            epochs = 1
        )
        del samples_noisy;
        
    xval_noisy = xval.copy()
    xval_noisy+=(noise_factor * np.random.normal(loc=0.0, scale=1.0, size = xval_noisy.shape))
    val_loss, val_acc = model.evaluate(xval_noisy, xval_noisy, xvalsteps=1)
    del xval_noisy
    
    if val_loss < lowest_val_loss:
        print('val_loss:{}'.format(val_loss))
        lowest_val_loss = val_loss
        cpsavepath = 'checkpointsandlogs/deepautoencoder/checkpoints/false_'+'fold_{}____'.format(f)+datetime.now().strftime("%d_%m_%Y____%H_%M_%S")
        with open(csavepath+'.json', "w") as json_file:
            json_file.write(model.to_json())
        model.save_weights(csavepath+'.h5')
        
    
        
        
        

In [ ]:
a = autoencoder.predict(np.expand_dims(img_to_array(load_img(trainfalsedirpath + '/' + trainfalsedir[21])), axis = 0))

In [ ]:
from keras import backend
backend.clear_session()